# The aff3ct 'hello world'

This tutorial presents the steps for playing with `aff3ct`. 

Before going farther, we should import the `py_aff3ct`library. The sys library is requiered to add the library path to your path. For this example, we will also need the `numpy` library.

In [44]:
import sys  
sys.path.insert(0, '../build')

import py_aff3ct

import numpy as np
import math

The `py_aff3ct` library is composed of `modules`. Globally, a module is a device that can process data. A data processing is called a `task` in the `aff3ct` language. Tasks exchange data through `sockets`.

Let start by building a source for generating random binary sequences of length 16 and displaying its `tasks` and `sockets`.

In [48]:
src = py_aff3ct.module.source.Source_random(16)
src.info()

-----------------------------Source_random-----------------------------
- Name         : 
- Class        : Source_random
- Frames number: 1
- Tasks        :
	- Task 0
		- Name         : generate
		- Sockets      : 
			- Socket 0
				- Name              : U_K
				- Type              : out
				- Elements per frame: 16
				- Data type         : int32
			- Socket 1
				- Name              : status
				- Type              : out
				- Elements per frame: 1
				- Data type         : int32



Now we can generate the bits by executing the task named `generate`.

In [49]:
src('generate').exec()
bits = src['generate::U_K'][:]
print(bits)

[[0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0]]


The second line shows how to access the bits within your python code. The variable named `bits` is a NumPy array.

We want now to modulate `U_K` using BPSK modulation. So, we continue by building a BPSK `modem` (which is a also a `module`).

In [47]:
mod = py_aff3ct.module.modem.Modem_BPSK(16)
mod.info()

-----------------------------Modem_BPSK-----------------------------
- Name         : 
- Class        : Modem_BPSK
- Frames number: 1
- Tasks        :
	- Task 0
		- Name         : modulate
		- Sockets      : 
			- Socket 0
				- Name              : X_N1
				- Type              : in
				- Elements per frame: 16
				- Data type         : int32
			- Socket 1
				- Name              : X_N2
				- Type              : out
				- Elements per frame: 16
				- Data type         : float32
			- Socket 2
				- Name              : status
				- Type              : out
				- Elements per frame: 1
				- Data type         : int32
	- Task 1
		- Name         : tmodulate
		- Sockets      : 
			- Socket 0
				- Name              : X_N1
				- Type              : in
				- Elements per frame: 16
				- Data type         : float32
			- Socket 1
				- Name              : X_N2
				- Type              : out
				- Elements per frame: 16
				- Data type         : float32
			- Socket 2
				- Name              : status


To link the source `src` and the modulator `mod`, the output socket `U_K` of the task `generate` of the module `src` should be "binded" to the input socket `X_N1` of the task `modulate` of the module `mod`.

In [38]:
 mod['modulate::X_N1'].bind(src['generate::U_K'])

Now we can execute the processing sequence.

In [41]:
src('generate').exec()
mod('modulate').exec()
bits    = src['generate::U_K'][:]
symbols = mod['modulate::X_N2'][:]
print("bits = ", bits)
print("symbols = ", symbols)

bits =  [[0 1 0 1 1 1 1 1 1 0 1 1 0 0 1 0]]
symbols =  [[ 1. -1.  1. -1. -1. -1. -1. -1. -1.  1. -1. -1.  1.  1. -1.  1.]]


You can see that the data is automatically transfered from the output of `generate` to the input of `modulate`. The data printing can be handled automatically by setting the `debug` property of the tasks to `true`.

In [43]:
src('generate').debug = True
mod('modulate').debug = True
src('generate').exec()
mod('modulate').exec()

# Source_random::generate(int32 U_K[16])
# {OUT} U_K = [    0,     1,     1,     0,     1,     0,     0,     1,     0,     0,     0,     1,     1,     0,     1,     0]
# Returned status: [0 'SUCCESS']
#
# Modem_BPSK::modulate(const int32 X_N1[16], float32 X_N2[16])
# {IN}  X_N1 = [    0,     1,     1,     0,     1,     0,     0,     1,     0,     0,     0,     1,     1,     0,     1,     0]
# {OUT} X_N2 = [ 1.00, -1.00, -1.00,  1.00, -1.00,  1.00,  1.00, -1.00,  1.00,  1.00,  1.00, -1.00, -1.00,  1.00, -1.00,  1.00]
# Returned status: [0 'SUCCESS']
#
